In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
import pandas as pd

In [ ]:
# Load your DataFrame
df = pd.read_csv('/content/drive/MyDrive/Research files/mydf_filtered.csv')
df

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,gold,0.665368,0.575129,-0.233913,-0.803415,0.541873,-0.284748,-0.587410,0.304050,-0.976940,...,0.664564,-0.326792,-0.166843,0.089043,0.174635,0.708083,-0.211870,-0.922929,0.107179,0.442703
1,mines,-0.061983,-0.611545,0.075108,-1.006732,1.258891,-0.059213,-1.300372,-0.547770,0.142394,...,-0.045271,-0.081380,0.522956,-0.335626,0.570412,0.794796,-0.023208,0.021023,0.607977,-0.573830
2,creek,-0.047574,0.458035,0.221478,-0.578259,0.613073,-0.846475,-0.203815,1.112873,-1.040184,...,-0.104678,-0.689773,-0.158615,0.059416,1.047682,0.612500,0.655737,0.208093,-0.650601,-0.055082
3,work,0.488443,-0.393066,0.145720,-0.665517,0.423911,-0.420992,-0.329604,0.477476,-0.274982,...,-0.463748,0.348744,-0.168836,-0.793235,-0.274093,0.636341,0.018944,0.415261,0.409543,0.393351
4,copper,-0.012102,0.202483,0.003206,-0.243465,1.527658,-0.297196,-0.035213,0.253279,-0.732782,...,0.117608,-0.238600,-0.178582,-0.869530,-0.214733,0.395891,0.251760,0.630467,0.392041,0.138883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12062,xanthate,-0.031826,0.257864,-0.054609,0.185644,-0.256702,0.007058,0.342248,0.009022,-0.007677,...,0.030342,0.134292,0.025772,0.166147,-0.060410,-0.262042,-0.027989,0.021065,-0.031498,-0.007414
12063,yauco,-0.129316,0.196438,0.083277,0.112372,-0.092597,-0.162406,0.167547,0.049535,-0.147225,...,0.039223,0.048401,-0.136274,0.303611,-0.034266,-0.222302,-0.050786,0.147664,0.082796,-0.038463
12064,yearbook,0.161926,-0.127037,-0.179428,0.064139,0.039060,0.316847,-0.100566,-0.017685,-0.085215,...,0.061324,-0.095374,0.105686,0.122895,0.311127,0.216142,-0.028143,0.326211,-0.248389,-0.068676
12065,yest,0.113457,-0.214988,0.029493,0.070235,0.015440,-0.153996,0.007843,-0.117899,-0.028560,...,0.016367,0.091974,-0.056424,0.105577,-0.001417,-0.104674,-0.058605,-0.099760,0.224277,-0.158745


In [ ]:
words = df.iloc[:, 0].values
embeddings = df.iloc[:, 1:].values

# Convert embeddings to tensor
embeddings_tensor = torch.tensor(embeddings, dtype=torch.float32)

# Create TensorDataset
dataset = TensorDataset(embeddings_tensor)

# Split into train and test datasets
train_size = int(0.8 * len(dataset))  # 80% training data
test_size = len(dataset) - train_size  # 20% test data
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for training and testing
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# class LSTM_VAE(nn.Module):
#     def __init__(self, input_dim, hidden_dims, latent_dim):
#         super(LSTM_VAE, self).__init__()

#         self.hidden_dims = hidden_dims

#         # Encoder
#         self.lstm_encoder = nn.LSTM(input_dim, hidden_dims[0], batch_first=True)
#         self.fc_enc = nn.Sequential(
#             nn.Linear(hidden_dims[0], hidden_dims[1]),
#             nn.ReLU(),
#             nn.Linear(hidden_dims[1], hidden_dims[2]),
#             nn.ReLU(),
#             nn.Linear(hidden_dims[2], hidden_dims[3]),
#             nn.ReLU(),
#             nn.Linear(hidden_dims[3], hidden_dims[4]),
#             nn.ReLU()
#         )
#         self.fc_mu = nn.Linear(hidden_dims[4], latent_dim)
#         self.fc_logvar = nn.Linear(hidden_dims[4], latent_dim)

#         # Decoder
#         self.fc_dec = nn.Sequential(
#             nn.Linear(latent_dim, hidden_dims[4]),
#             nn.ReLU(),
#             nn.Linear(hidden_dims[4], hidden_dims[3]),
#             nn.ReLU(),
#             nn.Linear(hidden_dims[3], hidden_dims[2]),
#             nn.ReLU(),
#             nn.Linear(hidden_dims[2], hidden_dims[1]),
#             nn.ReLU(),
#             nn.Linear(hidden_dims[1], hidden_dims[0]),
#             nn.ReLU()
#         )
#         self.lstm_decoder = nn.LSTM(hidden_dims[0], input_dim, batch_first=True)

#     def encode(self, x):
#         lstm_out, _ = self.lstm_encoder(x)
#         lstm_out = lstm_out[:, -1, :]  # Take the output of the last time step
#         enc_out = self.fc_enc(lstm_out)
#         mu = self.fc_mu(enc_out)
#         logvar = self.fc_logvar(enc_out)
#         return mu, logvar

#     def reparameterize(self, mu, logvar):
#         std = torch.exp(0.5 * logvar)
#         eps = torch.randn_like(std)
#         return mu + eps * std

#     def decode(self, z):
#         dec_out = self.fc_dec(z)
#         hidden = dec_out.unsqueeze(1)  # Reshape for LSTM
#         lstm_out, _ = self.lstm_decoder(hidden)
#         return lstm_out.squeeze(1)

#     def forward(self, x):
#         mu, logvar = self.encode(x)
#         z = self.reparameterize(mu, logvar)
#         x_reconstructed = self.decode(z)
#         return x_reconstructed, mu, logvar

In [ ]:
class LSTM_VAE(nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim):
        super(LSTM_VAE, self).__init__()

        self.lstm_encoder = nn.LSTM(input_dim, hidden_dims[0], batch_first=True)
        self.lstm_enc = nn.LSTM(hidden_dims[0], hidden_dims[1], batch_first=True)
        self.lstm_enc2 = nn.LSTM(hidden_dims[1], hidden_dims[2], batch_first=True)
        self.lstm_enc3 = nn.LSTM(hidden_dims[2], hidden_dims[3], batch_first=True)
        self.lstm_enc4 = nn.LSTM(hidden_dims[3], hidden_dims[4], batch_first=True)

        self.fc_mu = nn.Linear(hidden_dims[4], latent_dim)
        self.fc_logvar = nn.Linear(hidden_dims[4], latent_dim)

        self.fc_dec = nn.Linear(latent_dim, hidden_dims[4])
        self.lstm_dec4 = nn.LSTM(hidden_dims[4], hidden_dims[3], batch_first=True)
        self.lstm_dec3 = nn.LSTM(hidden_dims[3], hidden_dims[2], batch_first=True)
        self.lstm_dec2 = nn.LSTM(hidden_dims[2], hidden_dims[1], batch_first=True)
        self.lstm_dec1 = nn.LSTM(hidden_dims[1], hidden_dims[0], batch_first=True)
        self.lstm_dec = nn.LSTM(hidden_dims[0], input_dim, batch_first=True)

        self.relu = nn.ReLU()

    def encode(self, x):
        lstm_out, _ = self.lstm_encoder(x)
        lstm_out, _ = self.lstm_enc(lstm_out)
        lstm_out, _ = self.lstm_enc2(lstm_out)
        lstm_out, _ = self.lstm_enc3(lstm_out)
        lstm_out, _ = self.lstm_enc4(lstm_out)
        lstm_out = lstm_out[:, -1, :]  # Take the output of the last time step
        mu = self.fc_mu(lstm_out)
        logvar = self.fc_logvar(lstm_out)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, seq_len):
        # Pass through the fully connected layer
        dec_out = self.fc_dec(z)  # [batch_size, hidden_dims[4]]
        dec_out = self.relu(dec_out)

        # Repeat the latent vector for each time step
        dec_out = dec_out.unsqueeze(1).repeat(1, seq_len, 1)  # [batch_size, seq_len, hidden_dims[4]]

        # Pass through the decoder LSTM layers
        lstm_out, _ = self.lstm_dec4(dec_out)  # [batch_size, seq_len, hidden_dims[3]]
        # lstm_out = self.relu(lstm_out)

        lstm_out, _ = self.lstm_dec3(lstm_out)  # [batch_size, seq_len, hidden_dims[2]]
        # lstm_out = self.relu(lstm_out)

        lstm_out, _ = self.lstm_dec2(lstm_out)  # [batch_size, seq_len, hidden_dims[1]]
        # lstm_out = self.relu(lstm_out)

        lstm_out, _ = self.lstm_dec1(lstm_out)  # [batch_size, seq_len, hidden_dims[0]]
        # lstm_out = self.relu(lstm_out)

        lstm_out, _ = self.lstm_dec(lstm_out)  # [batch_size, seq_len, input_dim]

        return lstm_out

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        seq_len = x.size(1)  # Assuming x is of shape [batch_size, seq_len, input_dim]
        x_reconstructed = self.decode(z, seq_len)
        return x_reconstructed, mu, logvar

In [ ]:
# Loss function
def loss_function(recon_x, x, mu, logvar):
    MSE = nn.functional.mse_loss(recon_x, x, reduction = 'sum')
    # KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    KLD = 0
    return MSE + KLD

In [ ]:
# Model parameters
input_dim = embeddings.shape[1]  # Adjust as per your data
hidden_dims = [128, 64, 32, 16, 8]
latent_dim = 2

# Instantiate model
model = LSTM_VAE(input_dim, hidden_dims, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Training parameters
num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch in train_loader:
        data = batch[0]

        # Ensure data is properly shaped for LSTM input
        data = data.unsqueeze(1)  # Add sequence length dimension

        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)

        # Ensure recon_batch is of the same shape as data
        recon_batch = recon_batch.view_as(data)

        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    # Calculate average loss for the training epoch
    average_train_loss = train_loss / len(train_loader.dataset)

    # Evaluate the model on the test set
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Disable gradient computation during testing
        for batch in test_loader:
            data = batch[0]

            # Ensure data is properly shaped for LSTM input
            data = data.unsqueeze(1)  # Add sequence length dimension

            recon_batch, mu, logvar = model(data)

            # Ensure recon_batch is of the same shape as data
            recon_batch = recon_batch.view_as(data)

            loss = loss_function(recon_batch, data, mu, logvar)
            test_loss += loss.item()

    # Calculate average loss for the test epoch
    average_test_loss = test_loss / len(test_loader.dataset)

    print(f'Epoch {epoch + 1}/{num_epochs},Train Loss: {average_train_loss}, Test Loss: {average_test_loss}')

Epoch 1/30,Train Loss: 16.673434861445074, Test Loss: 16.834945590217547
Epoch 2/30,Train Loss: 16.234975664847553, Test Loss: 16.547046392736302
Epoch 3/30,Train Loss: 16.060795666133384, Test Loss: 16.43226831494626
Epoch 4/30,Train Loss: 15.948946216298326, Test Loss: 16.32333782989325
Epoch 5/30,Train Loss: 15.862034480633247, Test Loss: 16.241788982653677
Epoch 6/30,Train Loss: 15.793352115022127, Test Loss: 16.183559673723348
Epoch 7/30,Train Loss: 15.740928922578311, Test Loss: 16.134631190896528
Epoch 8/30,Train Loss: 15.68253641218444, Test Loss: 16.08488447920982
Epoch 9/30,Train Loss: 15.641879666695061, Test Loss: 16.03423382017172
Epoch 10/30,Train Loss: 15.609305169660663, Test Loss: 16.021424180530968
Epoch 11/30,Train Loss: 15.574662236516295, Test Loss: 16.019922112667967
Epoch 12/30,Train Loss: 15.56146459323587, Test Loss: 16.008285731052503
Epoch 13/30,Train Loss: 15.558553643712203, Test Loss: 16.001504002075702
Epoch 14/30,Train Loss: 15.529115094183195, Test Loss

In [ ]:
# Add the sequence length dimension (which is 1 in this case)
embeddings_tensor = embeddings_tensor.unsqueeze(1)  # Shape: [batch_size, sequence_length (1), input_dim]

# Forward pass through the encoder to get latent vectors
with torch.no_grad():
    mu, _ = model.encode(embeddings_tensor)
    latent_vectors = model.reparameterize(mu, torch.zeros_like(mu))  # Use zero variance for deterministic latent vectors

In [ ]:
# Convert latent vectors to numpy array
latent_vectors_np = latent_vectors.numpy()

# Create a DataFrame
latent_df = pd.DataFrame(latent_vectors_np, columns=[f'LatentDim_{i+1}' for i in range(latent_vectors_np.shape[1])])
latent_df['Word'] = words

# Reorder columns to have words first
latent_df = latent_df[['Word'] + [f'LatentDim_{i+1}' for i in range(latent_vectors_np.shape[1])]]

latent_df

,Word,LatentDim_1,LatentDim_2
0,gold,-0.712100,0.053557
1,mines,0.249579,-1.114365
2,creek,1.750212,-0.644575
3,work,2.163980,1.457885
4,copper,2.575934,-2.049365
...,...,...,...
12062,xanthate,2.784991,-3.414937
12063,yauco,3.131570,-6.296589
12064,yearbook,3.815028,-1.891092
12065,yest,4.187235,-2.478442


In [ ]:

# # Save to CSV file
# latent_df.to_csv('/content/drive/MyDrive/Research files/VAE_LSTM_reduced_data.csv', index=False)
# latent_df

,Word,LatentDim_1,LatentDim_2
0,gold,-0.712100,0.053557
1,mines,0.249579,-1.114365
2,creek,1.750212,-0.644575
3,work,2.163980,1.457885
4,copper,2.575934,-2.049365
...,...,...,...
12062,xanthate,2.784991,-3.414937
12063,yauco,3.131570,-6.296589
12064,yearbook,3.815028,-1.891092
12065,yest,4.187235,-2.478442


In [ ]:
class LSTM_VAE(nn.Module):
    def __init__(self, input_dim, hidden_dims, latent_dim):
        super(LSTM_VAE, self).__init__()

        self.lstm_encoder = nn.LSTM(input_dim, hidden_dims[0], batch_first=True)
        self.lstm_enc = nn.LSTM(hidden_dims[0], hidden_dims[1], batch_first=True)
        self.lstm_enc2 = nn.LSTM(hidden_dims[1], hidden_dims[2], batch_first=True)

        self.fc_mu = nn.Linear(hidden_dims[2], latent_dim)
        self.fc_logvar = nn.Linear(hidden_dims[2], latent_dim)

        self.fc_dec = nn.Linear(latent_dim, hidden_dims[2])
        self.lstm_dec2 = nn.LSTM(hidden_dims[2], hidden_dims[1], batch_first=True)
        self.lstm_dec1 = nn.LSTM(hidden_dims[1], hidden_dims[0], batch_first=True)
        self.lstm_dec = nn.LSTM(hidden_dims[0], input_dim, batch_first=True)

        self.relu = nn.ReLU()

    def encode(self, x):
        lstm_out, _ = self.lstm_encoder(x)
        lstm_out, _ = self.lstm_enc(lstm_out)
        lstm_out, _ = self.lstm_enc2(lstm_out)
        lstm_out = lstm_out[:, -1, :]  # Take the output of the last time step
        mu = self.fc_mu(lstm_out)
        logvar = self.fc_logvar(lstm_out)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, seq_len):
        # Pass through the fully connected layer
        dec_out = self.fc_dec(z)  # [batch_size, hidden_dims[4]]
        dec_out = self.relu(dec_out)

        # Repeat the latent vector for each time step
        dec_out = dec_out.unsqueeze(1).repeat(1, seq_len, 1)  # [batch_size, seq_len, hidden_dims[4]]


        lstm_out, _ = self.lstm_dec2(dec_out)  # [batch_size, seq_len, hidden_dims[1]]
        # lstm_out = self.relu(lstm_out)

        lstm_out, _ = self.lstm_dec1(lstm_out)  # [batch_size, seq_len, hidden_dims[0]]
        # lstm_out = self.relu(lstm_out)

        lstm_out, _ = self.lstm_dec(lstm_out)  # [batch_size, seq_len, input_dim]

        return lstm_out

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        seq_len = x.size(1)  # Assuming x is of shape [batch_size, seq_len, input_dim]
        x_reconstructed = self.decode(z, seq_len)
        return x_reconstructed, mu, logvar

In [ ]:
latent_dim = 16

# Instantiate model
model = LSTM_VAE(input_dim, hidden_dims, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Training parameters
num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch in train_loader:
        data = batch[0]

        # Ensure data is properly shaped for LSTM input
        data = data.unsqueeze(1)  # Add sequence length dimension

        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)

        # Ensure recon_batch is of the same shape as data
        recon_batch = recon_batch.view_as(data)

        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    # Calculate average loss for the training epoch
    average_train_loss = train_loss / len(train_loader.dataset)

    # Evaluate the model on the test set
    model.eval()
    test_loss = 0
    with torch.no_grad():  # Disable gradient computation during testing
        for batch in test_loader:
            data = batch[0]

            # Ensure data is properly shaped for LSTM input
            data = data.unsqueeze(1)  # Add sequence length dimension

            recon_batch, mu, logvar = model(data)

            # Ensure recon_batch is of the same shape as data
            recon_batch = recon_batch.view_as(data)

            loss = loss_function(recon_batch, data, mu, logvar)
            test_loss += loss.item()

    # Calculate average loss for the test epoch
    average_test_loss = test_loss / len(test_loader.dataset)

    print(f'Epoch {epoch + 1}/{num_epochs},Train Loss: {average_train_loss}, Test Loss: {average_test_loss}')

Epoch 1/30,Train Loss: 16.349410585525753, Test Loss: 16.463974877636396


KeyboardInterrupt: 

**Extract the latent vector**
-
-
- Since we already have separated embedding table
-
-

In [ ]:
# Add the sequence length dimension (which is 1 in this case)
embeddings_tensor = embeddings_tensor.unsqueeze(1)  # Shape: [batch_size, sequence_length (1), input_dim]

# Forward pass through the encoder to get latent vectors
with torch.no_grad():
    mu, _ = model.encode(embeddings_tensor)
    latent_vectors = model.reparameterize(mu, torch.zeros_like(mu))  # Use zero variance for deterministic latent vectors

In [ ]:
# Convert latent vectors to numpy array
latent_vectors_np = latent_vectors.numpy()

# Create a DataFrame
latent_df = pd.DataFrame(latent_vectors_np, columns=[f'LatentDim_{i+1}' for i in range(latent_vectors_np.shape[1])])
latent_df['Word'] = words

# Reorder columns to have words first
latent_df = latent_df[['Word'] + [f'LatentDim_{i+1}' for i in range(latent_vectors_np.shape[1])]]

In [ ]:
latent_df

In [ ]:
# Load your DataFrame
cities_df = pd.read_csv('/content/drive/MyDrive/Research files/worldcities.csv')
cities_df['city'] = cities_df['city'].str.lower()

city_names = cities_df['city'].values

cities_df

In [ ]:
len(city_names)

In [ ]:
# Identify the rows corresponding to city names
encoded_city_df = latent_df[latent_df['Word'].isin(city_names)]
encoded_city_df

In [ ]:
import matplotlib.pyplot as plt

# Plot the 2D embeddings
plt.figure(figsize=(10, 8))
plt.scatter(latent_df['LatentDim_1'], latent_df['LatentDim_2'], alpha=0.5, label='Word')
plt.scatter(encoded_city_df['LatentDim_1'], encoded_city_df['LatentDim_2'], color='red', label='Cities')
plt.legend()
plt.xlabel('Latent Dimension 1')
plt.ylabel('Latent Dimension 2')
plt.title('2D Visualization of Word Embeddings with Cities Highlighted')
plt.show()